# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
issue_d_month = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
}
next_pymnt_d_month = {
    "Apr-2019": 4,
    "May-2019": 5,
}

In [5]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Months' names encoded using the dictionary values
df["issue_d_month"] = df["issue_d"].apply(lambda x: issue_d_month[x])
df.drop(["issue_d"], axis=1, inplace=True)

df["next_pymnt_d_month"] = df["next_pymnt_d"].apply(lambda x: next_pymnt_d_month[x])
df.drop(["next_pymnt_d"], axis=1, inplace=True)

df = pd.get_dummies(df, columns=["home_ownership","verification_status","pymnt_plan","initial_list_status","application_type","hardship_flag","debt_settlement_flag"])

## Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [7]:
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0,0,1,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0,0,1,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,0,1,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,0,1,0,1,1,0,1,1


In [9]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(51612, 92)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()


In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
data_scaler.fit(X_train)


StandardScaler()

In [13]:
# Scale the training and testing data
X_train = data_scaler.transform(X_train)
X_test = data_scaler.transform(X_test)


## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7637493906394157

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   55,    32],
       [ 1792, 15326]])

In [17]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.63      0.90      0.06      0.75      0.55        87
   low_risk       1.00      0.90      0.63      0.94      0.75      0.58     17118

avg / total       0.99      0.89      0.63      0.94      0.75      0.58     17205



In [18]:
# List the features sorted in descending order by feature importance
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
         axis=0)
indices = np.argsort(importances)[::-1]
print(indices)


[16 13 15 20 14  1 76  4  2 35 53 44 48 73 47 43  9 25 74 36 42 32  3 30
 12 31 72 11  0 75 50 61 52 60 37 58 10 59 49 63 68 41 39 55 57 17  7 40
 38 62 29 51 56 67  6 27 69 34 28 33 77 26 24  5 54 84 89 82 80 83 70 81
 88  8 86 79 66 87 21 78 45 46 18 19 22 23 90 64 65 71 85 91]


In [19]:
Column_list = X.columns.tolist()

In [20]:
for i in indices:
    print(Column_list[i])

total_rec_int
total_pymnt
total_rec_prncp
last_pymnt_amnt
total_pymnt_inv
int_rate
issue_d_month
dti
installment
max_bal_bc
mths_since_recent_inq
bc_util
mo_sin_old_rev_tl_op
total_bal_ex_mort
mo_sin_old_il_acct
bc_open_to_buy
revol_bal
tot_cur_bal
total_bc_limit
all_util
avg_cur_bal
il_util
annual_inc
mths_since_rcnt_il
out_prncp_inv
total_bal_il
tot_hi_cred_lim
out_prncp
loan_amnt
total_il_high_credit_limit
mo_sin_rcnt_tl
num_rev_accts
mths_since_recent_bc
num_op_rev_tl
total_rev_hi_lim
num_bc_tl
total_acc
num_il_tl
mo_sin_rcnt_rev_tl_op
num_sats
pct_tl_nvr_dlq
acc_open_past_24mths
total_cu_tl
num_actv_bc_tl
num_bc_sats
total_rec_late_fee
open_acc
inq_last_12m
inq_fi
num_rev_tl_bal_gt_0
open_il_24m
mort_acc
num_actv_rev_tl
num_tl_op_past_12m
inq_last_6mths
open_act_il
percent_bc_gt_75
open_rv_24m
open_il_12m
open_rv_12m
next_pymnt_d_month
open_acc_6m
tot_coll_amt
delinq_2yrs
num_accts_ever_120_pd
verification_status_Verified
application_type_Joint App
verification_status_Not Verified

### Easy Ensemble Classifier

In [21]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
model = EasyEnsembleClassifier(n_estimators=100,random_state=1)
model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [22]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9251936860171386

In [23]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  987, 16131]])

In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    EasyEnsembleClassifier

2. Which model had the best recall score?

    EasyEnsembleClassifier

3. Which model had the best geometric mean score?

    EasyEnsembleClassifier

4. What are the top three features?

    total_rec_int,
    total_pymnt,
    total_rec_prncp,